<script async src="https://www.googletagmanager.com/gtag/js?id=UA-59152712-8"></script>
<script>
  window.dataLayer = window.dataLayer || [];
  function gtag(){dataLayer.push(arguments);}
  gtag('js', new Date());

  gtag('config', 'UA-59152712-8');
</script>

# Tutorial-IllinoisGRMHD__ID_converter

## Authors: Leo Werneck & Zach Etienne

<font color='red'>**This module is currently under development**</font>

## In this tutorial module we generate the initial data conversion routine, which will translate ETK quantities into quantities used by IllinoisGRMHD

### Required and recommended citations:

* **(Required)** Etienne, Z. B., Paschalidis, V., Haas R., Mösta P., and Shapiro, S. L. IllinoisGRMHD: an open-source, user-friendly GRMHD code for dynamical spacetimes. Class. Quantum Grav. 32 (2015) 175009. ([arxiv:1501.07276](http://arxiv.org/abs/1501.07276)).
* **(Required)** Noble, S. C., Gammie, C. F., McKinney, J. C., Del Zanna, L. Primitive Variable Solvers for Conservative General Relativistic Magnetohydrodynamics. Astrophysical Journal, 641, 626 (2006) ([astro-ph/0512420](https://arxiv.org/abs/astro-ph/0512420)).
* **(Recommended)** Del Zanna, L., Bucciantini N., Londrillo, P. An efficient shock-capturing central-type scheme for multidimensional relativistic flows - II. Magnetohydrodynamics. A&A 400 (2) 397-413 (2003). DOI: 10.1051/0004-6361:20021641 ([astro-ph/0210618](https://arxiv.org/abs/astro-ph/0210618)).

In [1]:
%%writefile ../src/set_IllinoisGRMHD_metric_GRMHD_variables_based_on_HydroBase_and_ADMBase_variables.C
/********************************
 * CONVERT ET ID TO IllinoisGRMHD
 * 
 * Written in 2014 by Zachariah B. Etienne
 *
 * Sets metric & MHD variables needed 
 * by IllinoisGRMHD, converting from
 * HydroBase and ADMBase.
 ********************************/

#include <stdio.h>
#include <stdlib.h>
#include <math.h>
#include <sys/time.h>
#include "cctk.h"
#include "cctk_Parameters.h"
#include "cctk_Arguments.h"
#include "IllinoisGRMHD_headers.h"

extern "C" void set_IllinoisGRMHD_metric_GRMHD_variables_based_on_HydroBase_and_ADMBase_variables(CCTK_ARGUMENTS) {

  DECLARE_CCTK_ARGUMENTS;
  DECLARE_CCTK_PARAMETERS;

  if(rho_b_atm > 1e199) {
    CCTK_VError(VERR_DEF_PARAMS, "You MUST set rho_b_atm to some reasonable value in your param.ccl file.\n");
  }

  // Overwrite the metric with "random", constant
  // values everywhere, for debugging purposes
#pragma omp parallel for
  for(int k=0;k<cctk_lsh[2];k++)
    for(int j=0;j<cctk_lsh[1];j++)
      for(int i=0;i<cctk_lsh[0];i++) {
        int index=CCTK_GFINDEX3D(cctkGH,i,j,k);
        gxx[index]   = 1.300;
        gxy[index]   = 0.123;
        gxz[index]   = 0.0512;
        gyy[index]   = 1.100;
        gyz[index]   = 0.145;
        gzz[index]   = 1.111;
        alp[index]   = 0.4;
        rho[index]   = 0.1;
        betax[index] = 1e-2;
        betay[index] = 2e-2;
        betaz[index] = 3e-2;
        press[index] = 0.04;
        vel[CCTK_GFINDEX4D(cctkGH,i,j,k,0)] = -1.5e-2;
        vel[CCTK_GFINDEX4D(cctkGH,i,j,k,1)] = -1.1e-2;
        vel[CCTK_GFINDEX4D(cctkGH,i,j,k,2)] = -1.05e-2;
      }
    
  // Convert ADM variables (from ADMBase) to the BSSN-based variables expected by this routine.
  IllinoisGRMHD_convert_ADM_to_BSSN__enforce_detgtij_eq_1__and_compute_gtupij(cctkGH,cctk_lsh,  gxx,gxy,gxz,gyy,gyz,gzz,alp,
                                                                              gtxx,gtxy,gtxz,gtyy,gtyz,gtzz,
                                                                              gtupxx,gtupxy,gtupxz,gtupyy,gtupyz,gtupzz,
                                                                              phi_bssn,psi_bssn,lapm1);

  /***************
   * PPEOS Patch *
   ***************/
  eos_struct eos;
  initialize_EOS_struct_from_input(eos);
  
  if(pure_hydro_run) {
#pragma omp parallel for
    for(int k=0;k<cctk_lsh[2];k++) for(int j=0;j<cctk_lsh[1];j++) for(int i=0;i<cctk_lsh[0];i++) {
          int index=CCTK_GFINDEX3D(cctkGH,i,j,k);
          Avec[CCTK_GFINDEX4D(cctkGH,i,j,k,0)]=0;
          Avec[CCTK_GFINDEX4D(cctkGH,i,j,k,1)]=0;
          Avec[CCTK_GFINDEX4D(cctkGH,i,j,k,2)]=0;
          Aphi[index]=0;
        }
  }

#pragma omp parallel for
  for(int k=0;k<cctk_lsh[2];k++) for(int j=0;j<cctk_lsh[1];j++) for(int i=0;i<cctk_lsh[0];i++) {
        int index=CCTK_GFINDEX3D(cctkGH,i,j,k);

        rho_b[index] = rho[index];
        P[index] = press[index];

        /***************
         * PPEOS Patch *
         ***************
         * We now verify that the initial data
         * provided by the user is indeed "cold",
         * i.e. it contains no Thermal part and
         * P = P_cold.
         */
        /* Compute P_cold */
        int polytropic_index = find_polytropic_K_and_Gamma_index(eos, rho_b[index]);
        CCTK_REAL K_poly     = eos.K_ppoly_tab[polytropic_index];
        CCTK_REAL Gamma_poly = eos.Gamma_ppoly_tab[polytropic_index];
        CCTK_REAL P_cold     = K_poly*pow(rho_b[index],Gamma_poly);

        /* Compare P and P_cold */
        /*
        CCTK_REAL P_rel_error = fabs(P[index] - P_cold)/P[index];
        if( rho_b[index] > rho_b_atm && P_rel_error > 1e-2 ) {

          // Determine the value of Gamma_poly_local associated with P[index]
          CCTK_VError(VERR_DEF_PARAMS,"Expected a piecewise polytropic EOS with local Gamma_poly = %.15e, but found a point such that Gamma_poly_local = %.15e.\nError = %e\nrho_b = %e\nrho_b_atm = %e\nP = %e\n",
                      Gamma_poly, 0.123456, P_rel_error, rho_b[index], rho_b_atm, P[index]);
        }
        */

        Ax[index] = Avec[CCTK_GFINDEX4D(cctkGH,i,j,k,0)];
        Ay[index] = Avec[CCTK_GFINDEX4D(cctkGH,i,j,k,1)];
        Az[index] = Avec[CCTK_GFINDEX4D(cctkGH,i,j,k,2)];
        psi6phi[index] = Aphi[index];
	
        CCTK_REAL ETvx = vel[CCTK_GFINDEX4D(cctkGH,i,j,k,0)];
        CCTK_REAL ETvy = vel[CCTK_GFINDEX4D(cctkGH,i,j,k,1)];
        CCTK_REAL ETvz = vel[CCTK_GFINDEX4D(cctkGH,i,j,k,2)];

        // IllinoisGRMHD defines v^i = u^i/u^0.
	
        // Meanwhile, the ET/HydroBase formalism, called the Valencia 
        // formalism, splits the 4 velocity into a purely spatial part
        // and a part that is normal to the spatial hypersurface:
        // u^a = G (n^a + U^a), (Eq. 14 of arXiv:1304.5544; G=W, U^a=v^a)
        // where n^a is the unit normal vector to the spatial hypersurface,
        // n_a = {-\alpha,0,0,0}, and U^a is the purely spatial part, which
        // is defined in HydroBase as the vel[] vector gridfunction.
        // Then u^a n_a = - \alpha u^0 = G n^a n_a = -G, and
        // of course \alpha u^0 = 1/sqrt(1+γ^ij u_i u_j) = \Gamma,
        // the standard Lorentz factor.

        // Note that n^i = - \beta^i / \alpha, so 
        // u^a = \Gamma (n^a + U^a) 
        // -> u^i = \Gamma ( U^i - \beta^i / \alpha )
        // which implies
        // v^i = u^i/u^0
        //     = \Gamma/u^0 ( U^i - \beta^i / \alpha ) <- \Gamma = \alpha u^0
        //     = \alpha ( U^i - \beta^i / \alpha )
        //     = \alpha U^i - \beta^i

        vx[index] = alp[index]*ETvx - betax[index];
        vy[index] = alp[index]*ETvy - betay[index];
        vz[index] = alp[index]*ETvz - betaz[index];

      }

  // Neat feature for debugging: Add a roundoff-error perturbation
  //    to the initial data.
  // Set random_pert variable to ~1e-14 for a random 15th digit
  //    perturbation.
  srand(random_seed); // Use srand() as rand() is thread-safe.
  for(int k=0;k<cctk_lsh[2];k++)
    for(int j=0;j<cctk_lsh[1];j++)
      for(int i=0;i<cctk_lsh[0];i++) {
        int index=CCTK_GFINDEX3D(cctkGH,i,j,k);
        CCTK_REAL pert = (random_pert*(CCTK_REAL)rand() / RAND_MAX);
        CCTK_REAL one_plus_pert=(1.0+pert);
        rho[index]*=one_plus_pert;
        vx[index]*=one_plus_pert;
        vy[index]*=one_plus_pert;
        vz[index]*=one_plus_pert;

        psi6phi[index]*=one_plus_pert;
        Ax[index]*=one_plus_pert;
        Ay[index]*=one_plus_pert;
        Az[index]*=one_plus_pert;
      }

  // Next compute B & B_stagger from A_i. Note that this routine also depends on
  //   the psi_bssn[] gridfunction being set to exp(phi).

  CCTK_REAL dxi = 1.0/CCTK_DELTA_SPACE(0);
  CCTK_REAL dyi = 1.0/CCTK_DELTA_SPACE(1);
  CCTK_REAL dzi = 1.0/CCTK_DELTA_SPACE(2);  

#pragma omp parallel for
  for(int k=0;k<cctk_lsh[2];k++)
    for(int j=0;j<cctk_lsh[1];j++)
      for(int i=0;i<cctk_lsh[0];i++) {
        // Look Mom, no if() statements!
        int shiftedim1 = (i-1)*(i!=0); // This way, i=0 yields shiftedim1=0 and shiftedi=1, used below for our COPY boundary condition.
        int shiftedi   = shiftedim1+1;

        int shiftedjm1 = (j-1)*(j!=0);
        int shiftedj   = shiftedjm1+1;

        int shiftedkm1 = (k-1)*(k!=0);
        int shiftedk   = shiftedkm1+1;

        int index,indexim1,indexjm1,indexkm1;

        int actual_index = CCTK_GFINDEX3D(cctkGH,i,j,k);

        CCTK_REAL Psi = psi_bssn[actual_index];
        CCTK_REAL Psim3 = 1.0/(Psi*Psi*Psi);

        // For the lower boundaries, the following applies a "copy" 
        //    boundary condition on Bi_stagger where needed.
        //    E.g., Bx_stagger(i,jmin,k) = Bx_stagger(i,jmin+1,k)
        //    We find the copy BC works better than extrapolation.
        // For the upper boundaries, we do the following copy:
        //    E.g., Psi(imax+1,j,k)=Psi(imax,j,k)
        /**************/
        /* Bx_stagger */
        /**************/

        index    = CCTK_GFINDEX3D(cctkGH,i,shiftedj,shiftedk);
        indexjm1 = CCTK_GFINDEX3D(cctkGH,i,shiftedjm1,shiftedk);
        indexkm1 = CCTK_GFINDEX3D(cctkGH,i,shiftedj,shiftedkm1);
        // Set Bx_stagger = \partial_y A_z - partial_z A_y
        // "Grid" Ax(i,j,k) is actually Ax(i,j+1/2,k+1/2)
        // "Grid" Ay(i,j,k) is actually Ay(i+1/2,j,k+1/2)
        // "Grid" Az(i,j,k) is actually Ay(i+1/2,j+1/2,k)
        // Therefore, the 2nd order derivative \partial_z A_y at (i+1/2,j,k) is:
        //          ["Grid" Ay(i,j,k) - "Grid" Ay(i,j,k-1)]/dZ
        Bx_stagger[actual_index] = (Az[index]-Az[indexjm1])*dyi - (Ay[index]-Ay[indexkm1])*dzi;

        // Now multiply Bx and Bx_stagger by 1/sqrt(gamma(i+1/2,j,k)]) = 1/sqrt(1/2 [gamma + gamma_ip1]) = exp(-6 x 1/2 [phi + phi_ip1] )
        int imax_minus_i = (cctk_lsh[0]-1)-i;
        int indexip1jk = CCTK_GFINDEX3D(cctkGH,i + ( (imax_minus_i > 0) - (0 > imax_minus_i) ),j,k);
        CCTK_REAL Psi_ip1 = psi_bssn[indexip1jk];
        Bx_stagger[actual_index] *= Psim3/(Psi_ip1*Psi_ip1*Psi_ip1);

        /**************/
        /* By_stagger */
        /**************/

        index    = CCTK_GFINDEX3D(cctkGH,shiftedi,j,shiftedk);
        indexim1 = CCTK_GFINDEX3D(cctkGH,shiftedim1,j,shiftedk);
        indexkm1 = CCTK_GFINDEX3D(cctkGH,shiftedi,j,shiftedkm1);
        // Set By_stagger = \partial_z A_x - \partial_x A_z
        By_stagger[actual_index] = (Ax[index]-Ax[indexkm1])*dzi - (Az[index]-Az[indexim1])*dxi;

        // Now multiply By and By_stagger by 1/sqrt(gamma(i,j+1/2,k)]) = 1/sqrt(1/2 [gamma + gamma_jp1]) = exp(-6 x 1/2 [phi + phi_jp1] )
        int jmax_minus_j = (cctk_lsh[1]-1)-j;
        int indexijp1k = CCTK_GFINDEX3D(cctkGH,i,j + ( (jmax_minus_j > 0) - (0 > jmax_minus_j) ),k);
        CCTK_REAL Psi_jp1 = psi_bssn[indexijp1k];
        By_stagger[actual_index] *= Psim3/(Psi_jp1*Psi_jp1*Psi_jp1);


        /**************/
        /* Bz_stagger */
        /**************/

        index    = CCTK_GFINDEX3D(cctkGH,shiftedi,shiftedj,k);
        indexim1 = CCTK_GFINDEX3D(cctkGH,shiftedim1,shiftedj,k);
        indexjm1 = CCTK_GFINDEX3D(cctkGH,shiftedi,shiftedjm1,k);
        // Set Bz_stagger = \partial_x A_y - \partial_y A_x
        Bz_stagger[actual_index] = (Ay[index]-Ay[indexim1])*dxi - (Ax[index]-Ax[indexjm1])*dyi;

        // Now multiply Bz_stagger by 1/sqrt(gamma(i,j,k+1/2)]) = 1/sqrt(1/2 [gamma + gamma_kp1]) = exp(-6 x 1/2 [phi + phi_kp1] )
        int kmax_minus_k = (cctk_lsh[2]-1)-k;
        int indexijkp1 = CCTK_GFINDEX3D(cctkGH,i,j,k + ( (kmax_minus_k > 0) - (0 > kmax_minus_k) ));
        CCTK_REAL Psi_kp1 = psi_bssn[indexijkp1];
        Bz_stagger[actual_index] *= Psim3/(Psi_kp1*Psi_kp1*Psi_kp1);

      }

#pragma omp parallel for
  for(int k=0;k<cctk_lsh[2];k++)
    for(int j=0;j<cctk_lsh[1];j++)
      for(int i=0;i<cctk_lsh[0];i++) {
        // Look Mom, no if() statements!
        int shiftedim1 = (i-1)*(i!=0); // This way, i=0 yields shiftedim1=0 and shiftedi=1, used below for our COPY boundary condition.
        int shiftedi   = shiftedim1+1;

        int shiftedjm1 = (j-1)*(j!=0);
        int shiftedj   = shiftedjm1+1;

        int shiftedkm1 = (k-1)*(k!=0);
        int shiftedk   = shiftedkm1+1;

        int index,indexim1,indexjm1,indexkm1;

        int actual_index = CCTK_GFINDEX3D(cctkGH,i,j,k);

        // For the lower boundaries, the following applies a "copy" 
        //    boundary condition on Bi and Bi_stagger where needed.
        //    E.g., Bx(imin,j,k) = Bx(imin+1,j,k)
        //    We find the copy BC works better than extrapolation.
        /******/
        /* Bx */
        /******/
        index = CCTK_GFINDEX3D(cctkGH,shiftedi,j,k);
        indexim1 = CCTK_GFINDEX3D(cctkGH,shiftedim1,j,k);
        // Set Bx = 0.5 ( Bx_stagger + Bx_stagger_im1 )
        // "Grid" Bx_stagger(i,j,k) is actually Bx_stagger(i+1/2,j,k)
        Bx[actual_index] = 0.5 * ( Bx_stagger[index] + Bx_stagger[indexim1] );

        /******/
        /* By */
        /******/
        index = CCTK_GFINDEX3D(cctkGH,i,shiftedj,k);
        indexjm1 = CCTK_GFINDEX3D(cctkGH,i,shiftedjm1,k);
        // Set By = 0.5 ( By_stagger + By_stagger_im1 )
        // "Grid" By_stagger(i,j,k) is actually By_stagger(i,j+1/2,k)
        By[actual_index] = 0.5 * ( By_stagger[index] + By_stagger[indexjm1] );

        /******/
        /* Bz */
        /******/
        index = CCTK_GFINDEX3D(cctkGH,i,j,shiftedk);
        indexkm1 = CCTK_GFINDEX3D(cctkGH,i,j,shiftedkm1);
        // Set Bz = 0.5 ( Bz_stagger + Bz_stagger_im1 )
        // "Grid" Bz_stagger(i,j,k) is actually Bz_stagger(i,j+1/2,k)
        Bz[actual_index] = 0.5 * ( Bz_stagger[index] + Bz_stagger[indexkm1] );
      }

  // Finally, enforce limits on primitives & compute conservative variables.
#pragma omp parallel for
  for(int k=0;k<cctk_lsh[2];k++)
    for(int j=0;j<cctk_lsh[1];j++)
      for(int i=0;i<cctk_lsh[0];i++) {
        static const int zero_int=0;
        int index = CCTK_GFINDEX3D(cctkGH,i,j,k);

        int ww;

        CCTK_REAL PRIMS[MAXNUMVARS];
        ww=0;
        PRIMS[ww] = rho_b[index]; ww++;
        PRIMS[ww] = P[index];     ww++;
        PRIMS[ww] = vx[index];    ww++;
        PRIMS[ww] = vy[index];    ww++;
        PRIMS[ww] = vz[index];    ww++;
        PRIMS[ww] = Bx[index];    ww++;
        PRIMS[ww] = By[index];    ww++;
        PRIMS[ww] = Bz[index];    ww++;

        CCTK_REAL CONF_METRIC[NUMVARS_FOR_CONF_METRIC];
#include "NRPy_generated_headers/read_in_CONF_METRIC_from_gridfunctions_with_index.h"

        CCTK_REAL ADM_3METRIC[NUMVARS_FOR_ADM_3METRIC];
#include "NRPy_generated_headers/ADM_3METRIC__alpha_beta_gammaDD__in_terms_of__CONF_METRIC.h"
#include "NRPy_generated_headers/compute__ADM_gammaUU_and_sqrtgamma__in_terms_of__ADM_3METRIC.h"

        CCTK_REAL CONSERVS[NUM_CONSERVS] = {0,0,0,0,0};
        CCTK_REAL g4dn[4][4];
        CCTK_REAL g4up[4][4];
        CCTK_REAL TUPMUNU[10],TDNMUNU[10];

          
        if( ( (i==0) && (j==0) & (k==0) ) || ( (i==14) && (j==14) & (k==14) ) ) {
            printf("pppp %d || %e %e %e %e %e\n",i,PRIMS[RHOB],PRIMS[PRESSURE],PRIMS[VX],PRIMS[VY],PRIMS[VZ]);
        }
 
        struct output_stats stats; stats.failure_checker=0;
        IllinoisGRMHD_enforce_limits_on_primitives_and_recompute_conservs(zero_int,PRIMS,stats,eos,
                                                                          ADM_3METRIC,g4dn,g4up,TUPMUNU,TDNMUNU,CONSERVS);
 
        if( ( (i==0) && (j==0) & (k==0) ) || ( (i==14) && (j==14) & (k==14) ) ) {
            printf("qqqq0 %d || %e %e %e %e %e\n",i,PRIMS[RHOB],PRIMS[PRESSURE],PRIMS[VX],PRIMS[VY],PRIMS[VZ]);
            printf("qqqq1 %d || %e %e %e %e %e\n",i,CONSERVS[RHOSTAR],CONSERVS[STILDEX],CONSERVS[STILDEY],CONSERVS[STILDEZ],CONSERVS[TAUENERGY]);
            printf("qqqq2 %d || %e %e %e %e %e %e %e %e %e %e\n",i,TUPMUNU[0],TUPMUNU[1],TUPMUNU[2],TUPMUNU[3],TUPMUNU[4],TUPMUNU[5],TUPMUNU[6],TUPMUNU[7],TUPMUNU[8],TUPMUNU[9]);
            printf("qqqq3 %d || %e %e %e %e %e %e %e %e %e %e\n",i,TDNMUNU[0],TDNMUNU[1],TDNMUNU[2],TDNMUNU[3],TDNMUNU[4],TDNMUNU[5],TDNMUNU[6],TDNMUNU[7],TDNMUNU[8],TDNMUNU[9]);
        }

        rho_b[index] = PRIMS[RHOB];
        P[index]     = PRIMS[PRESSURE];
        vx[index]    = PRIMS[VX];
        vy[index]    = PRIMS[VY];
        vz[index]    = PRIMS[VZ];

        rho_star[index] = CONSERVS[RHOSTAR];
        mhd_st_x[index] = CONSERVS[STILDEX];
        mhd_st_y[index] = CONSERVS[STILDEY];
        mhd_st_z[index] = CONSERVS[STILDEZ];
        tau[index]      = CONSERVS[TAUENERGY];

        if(update_Tmunu) {
          ww=0;
          eTtt[index] = TDNMUNU[ww]; ww++;
          eTtx[index] = TDNMUNU[ww]; ww++;
          eTty[index] = TDNMUNU[ww]; ww++;
          eTtz[index] = TDNMUNU[ww]; ww++;
          eTxx[index] = TDNMUNU[ww]; ww++;
          eTxy[index] = TDNMUNU[ww]; ww++;
          eTxz[index] = TDNMUNU[ww]; ww++;
          eTyy[index] = TDNMUNU[ww]; ww++;
          eTyz[index] = TDNMUNU[ww]; ww++;
          eTzz[index] = TDNMUNU[ww];
        }
      }
}
